In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_data = 5000   # number of data
num_vars = 5      # number of decision variables
num_ints = 5      # number of integer decision variables
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_train = np.random.uniform(1, 11, (train_size, num_vars)).astype(np.float32)
p_test = np.random.uniform(1, 11, (test_size, num_vars)).astype(np.float32)
p_dev = np.random.uniform(1, 11, (val_size, num_vars)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train}, name="train")
data_test = DictDataset({"p":p_test}, name="test")
data_dev = DictDataset({"p":p_dev}, name="dev")

In [6]:
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## NM Problem

In [7]:
import neuromancer as nm
from problem.neural import probQuadratic

def getNMProb(round_module):
    # parameters
    p = nm.constraint.variable("p")
    # variables
    x_bar = nm.constraint.variable("x_bar")
    x_rnd = nm.constraint.variable("x_rnd")

    # model
    obj_bar, constrs_bar = probQuadratic(x_bar, p, num_vars=num_vars, alpha=100)
    obj_rnd, constrs_rnd = probQuadratic(x_rnd, p, num_vars=num_vars, alpha=100)

    # define neural architecture for the solution mapping
    func = nm.modules.blocks.MLP(insize=num_vars, outsize=num_vars, bias=True,
                                 linear_map=nm.slim.maps["linear"], nonlin=nn.ReLU, hsizes=[80]*4)
    # solution map from model parameters: sol_map(p) -> x
    sol_map = nm.system.Node(func, ["p"], ["x_bar"], name="smap")

    # trainable components
    components = [sol_map, round_module]

    # penalty loss
    loss = nm.loss.PenaltyLoss(obj_rnd, constrs_rnd)
    problem = nm.problem.Problem(components, loss)

    return problem

## Exact Solver

In [8]:
from problem.solver import exactQuadratic
model = exactQuadratic(n_vars=num_vars, n_integers=num_ints)

In [9]:
objvals, conviols, elapseds = [], [], []
for p in tqdm(p_test):
    model.setParamValue(*p)
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:27<00:00, 36.23it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    200.432000          3.839661      0.027225
std      78.224654          5.579124      0.013338
min      36.000000          0.000000      0.016814
25%     149.000000          0.000000      0.023348
50%     190.000000          0.000000      0.026231
75%     247.000000          8.148394      0.029312
max     511.000000         25.352531      0.416190


## Heuristic

In [10]:
from heuristic import naive_round

In [11]:
# relaxed model
model_rel = model.relax()

In [12]:
sols, objvals, conviols, elapseds = [], [], [], []
for p in tqdm(p_test):
    model_rel.setParamValue(*p)
    tick = time.time()
    xval_init, _ = model_rel.solve("scip", max_iter=100)
    naive_round(xval_init, model)
    tock = time.time()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:49<00:00, 20.31it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  1000.00000       1000.000000   1000.000000
mean    179.96800         28.608444      0.048707
std      76.37923          7.851483      0.020005
min      26.00000         10.634741      0.017738
25%     130.00000         21.912093      0.031502
50%     170.00000         27.912093      0.049273
75%     222.00000         33.912093      0.059908
max     471.00000         53.912093      0.154011


## Learning to Round

In [13]:
from model.layer import netFC
from model.round import roundModel
# round x
layers_rnd = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundModel(layers=layers_rnd, param_keys=["p"], var_keys=["x_bar"], output_keys=["x_rnd"],
                        int_ind={"x_bar":model.intInd}, name="round")
problem = getNMProb(round_func)

In [14]:
# training
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 935.581787109375
epoch: 1  train_loss: 401.3537292480469
epoch: 2  train_loss: 377.9687805175781
epoch: 3  train_loss: 372.4937744140625
epoch: 4  train_loss: 373.88250732421875
epoch: 5  train_loss: 377.8916931152344
epoch: 6  train_loss: 377.7049865722656
epoch: 7  train_loss: 375.1439514160156
epoch: 8  train_loss: 373.6103210449219
epoch: 9  train_loss: 378.47021484375
epoch: 10  train_loss: 365.5278015136719
epoch: 11  train_loss: 364.9478454589844
epoch: 12  train_loss: 363.7472839355469
epoch: 13  train_loss: 361.2356262207031
epoch: 14  train_loss: 361.4027099609375
epoch: 15  train_loss: 359.34246826171875
epoch: 16  train_loss: 362.633056640625
epoch: 17  train_loss: 361.0362243652344
epoch: 18  train_loss: 355.6873779296875
epoch: 19  train_loss: 358.42816162109375
epoch: 20  train_loss: 359.5583801269531
epoch: 21  train_loss: 357.5403137207031
epoch: 22  train_loss: 349.9636535644531
epoch: 23  train_loss: 348.8908386230469
epoch: 24  train_loss: 347.

In [15]:
sols, objvals, conviols, elapseds = [], [], [], []
for p in tqdm(p_test):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 387.71it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    194.814000          8.442956      0.002324
std      65.672233          2.812250      0.000417
min      46.000000          2.650724      0.002031
25%     147.000000          6.302247      0.002119
50%     186.000000          8.211475      0.002192
75%     237.000000         10.302247      0.002362
max     442.000000         18.542614      0.006761


## Learnable Threshold

In [16]:
from model.layer import netFC
from model.threshold import roundThresholdModel
# round x
layers_rnd = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundThresholdModel(layers=layers_rnd, param_keys=["p"], var_keys=["x_bar"], output_keys=["x_rnd"],
                                 int_ind={"x_bar":model.intInd}, name="round")
problem = getNMProb(round_func)

In [17]:
# training
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 964.7532958984375
epoch: 1  train_loss: 389.64617919921875
epoch: 2  train_loss: 352.8912353515625
epoch: 3  train_loss: 348.14794921875
epoch: 4  train_loss: 344.8109436035156
epoch: 5  train_loss: 343.06121826171875
epoch: 6  train_loss: 344.32269287109375
epoch: 7  train_loss: 341.03192138671875
epoch: 8  train_loss: 339.51239013671875
epoch: 9  train_loss: 340.4180603027344
epoch: 10  train_loss: 338.1318664550781
epoch: 11  train_loss: 339.6825256347656
epoch: 12  train_loss: 338.2113037109375
epoch: 13  train_loss: 338.3124694824219
epoch: 14  train_loss: 337.4046325683594
epoch: 15  train_loss: 338.1555480957031
epoch: 16  train_loss: 336.8291931152344
epoch: 17  train_loss: 337.4896545410156
epoch: 18  train_loss: 336.86688232421875
epoch: 19  train_loss: 338.0745544433594
epoch: 20  train_loss: 336.8587646484375
epoch: 21  train_loss: 335.4654541015625
epoch: 22  train_loss: 336.9027099609375
epoch: 23  train_loss: 336.3321838378906
epoch: 24  train_loss:

In [18]:
sols, objvals, conviols, elapseds = [], [], [], []
for p in tqdm(p_test):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 404.99it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    212.501000          9.036044      0.002218
std      74.800756          3.197487      0.000274
min      46.000000          2.687708      0.001952
25%     158.000000          6.733402      0.002059
50%     204.500000          8.494876      0.002131
75%     260.000000         11.273997      0.002267
max     502.000000         20.109414      0.004355
